In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pygrib
from netCDF4 import Dataset

In [ ]:
path_to_model = "/mnt/data-weather/moddata/modgrb/"
path_to_satellite = "/mnt/data-weather/satdata/satnc/band2346/"
path_to_radar = "/mnt/data-weather/raddata/raddat/"

In [ ]:
path_to_model_output = "/mnt/data-weather/moddata/modpng/"
path_to_satellite_output = "/mnt/data-weather/satdata/satpng/band2346/"
path_to_radar_output = "/mnt/data-weather/raddata/radpng/"

In [ ]:
path_to_satellite = "/mnt/data-weather/corr/sat/band2/"
path_to_satellite_output = "/mnt/data-weather/corr/A2/"
directory = os.fsencode(path_to_satellite)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename)
    nc = Dataset(path_to_satellite+filename, 'r')
    #extracting the data we want from the file
    data = nc.variables['data'][:][0]
    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    nc.close()
    #print(data.shape, lat.shape, lon.shape)
    plt.figure(figsize=(10,10))
    m = Basemap(projection='mill',
            resolution=None,
            llcrnrlon=-102,
            urcrnrlon=-82,
            llcrnrlat=31,
            urcrnrlat=41)
    x,y = m(lon,lat)
    cs = m.pcolormesh(x,y,data,shading='flat',cmap="gray")
    plt.savefig(path_to_satellite_output+filename[0:-3]+".png",
                bbox_inches='tight',
                pad_inches=-0.08)
    plt.clf()
    plt.close()

In [ ]:
import os
from shutil import copy2
path_to_satellite = "/mnt/data-weather/satdata/satnc/band2346/"
path_to_other = "/mnt/data-weather/satdata/satnc/other/"
directory = os.fsencode(path_to_satellite)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename[18:20])
    if filename[18:20] == "15":
        copy2(path_to_satellite+filename, path_to_other+filename)

In [ ]:
from datetime import timedelta
import datetime
directory = os.fsencode(path_to_other)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    print(filename[12:15])
    date = datetime.datetime(2017, 1, 1) + datetime.timedelta(int(filename[12:15]) - 1)
    if (date.month <= 9 and date.day <= 9):
        #print("sat."+str(date.year)+"0"+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+"0"+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    elif (date.month <= 9 and date.day > 9):
        #print("sat."+str(date.year)+"0"+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+"0"+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    elif (date.month > 9 and date.day <= 9):
        #print("sat."+str(date.year)+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    else:
        #print("sat."+str(date.year)+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    os.rename(path_to_other+filename, path_to_other+name)

In [ ]:
sat = "/mnt/data-weather/satdata/satnc/other/"
rad = "/mnt/data-weather/raddata/raddat/"
sat_out = "/mnt/data-weather/corr/sat/"
rad_out = "/mnt/data-weather/corr/rad/"
sat_dir = os.fsencode(sat)
rad_dir = os.fsencode(rad)

for sat_file in os.listdir(sat_dir):
    sat_filename = os.fsdecode(sat_file)
    for rad_file in os.listdir(rad_dir):
        rad_filename = os.fsdecode(rad_file)
        #print(sat_filename[8:15], rad_filename[10:17])
        if sat_filename[8:15] == rad_filename[10:17]:
            copy2(sat+sat_filename, sat_out+sat_filename)
            copy2(rad+rad_filename, rad_out+rad_filename)

In [ ]:
sat = "/mnt/data-weather/corr/sat/"
sat_out = "/mnt/data-weather/corr/sat/band"
sat_dir = os.fsencode(sat)
for sat_file in os.listdir(sat_dir):
    file = os.fsdecode(sat_file)
    print(file)
    print(file[17:18])
    if len(file) > 5:
        if int(file[17:18]) == 2:
            os.rename(sat+file,sat_out+"2/"+file)
        if int(file[17:18]) == 3:
            os.rename(sat+file,sat_out+"3/"+file)   
        if int(file[17:18]) == 4:
            os.rename(sat+file,sat_out+"4/"+file)
        if int(file[17:18]) == 6:
            os.rename(sat+file,sat_out+"6/"+file)

In [ ]:
rad = "/mnt/data-weather/corr/rad/"
out = "/mnt/data-weather/corr/B2/"
directory = os.fsencode(rad)
dims = (126,201)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename)
    data = np.fromfile(rad+filename,dtype='int16',count=-1,sep='') # 2 byte integers
    #print('size of read vector',len(data))
    data = np.reshape(data,dims)
    #print(data)
    fig = plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(data, cmap='gray', origin='lower')
    fig.savefig(out+filename[0:-4]+".png",
                bbox_inches='tight',
                pad_inches=-0.36)
    plt.close(fig)